In [30]:
import sys
sys.path.append('../utils')

In [31]:
import pipeline
import importlib
import pandas as pd

importlib.reload(pipeline)
from pipeline import extract_session_data, clean_session_data, get_driver_laps, get_manual_finish_dict
from pipeline import assemble_race_dataset, extract_fp2_features, extract_quali_features, assemble_race_dataset_pre_race
from fastf1 import get_session

In [32]:
# Dutch GP 2025 (Round 15)
# This cell was made before the race so it excludes the race finish data 
df_fp2_15 = extract_session_data(2025, "Netherlands", "FP2")
df_quali_15 = extract_session_data(2025, "Netherlands", "Q")

fp2_clean_15 = clean_session_data(df_fp2_15)
quali_clean_15 = clean_session_data(df_quali_15)

fp2_features_15 = extract_fp2_features(fp2_clean_15)

quali_features_15 = extract_quali_features(quali_clean_15)
                       
race_df_15 = assemble_race_dataset_pre_race(fp2_features_15, quali_features_15)
race_df_15["grand_prix"] = "Netherlands"
race_df_15

core           INFO 	Loading data for Dutch Grand Prix - Practice 2 [v3.4.4]


req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastf1/core.py:1579: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<TimedeltaArray>
['0 days 00:38:39.086000']
Length: 1, dtype: timedelta64[ns]' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  result.loc[ma

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,92.309167,71.756,6,69.652,15,Netherlands
1,ALO,87.701083,69.977,12,69.366,8,Netherlands
2,ANT,87.443125,71.185,8,69.493,11,Netherlands
3,BEA,77.386529,71.113,17,70.262,19,Netherlands
4,BOR,78.776938,71.320,16,69.622,12,Netherlands
5,COL,85.379154,70.957,13,70.104,16,Netherlands
6,GAS,83.918000,72.122,17,69.637,14,Netherlands
7,HAM,87.312538,70.738,13,69.261,6,Netherlands
8,HUL,79.281437,71.080,16,70.195,17,Netherlands
9,LAW,85.178583,71.339,12,69.383,9,Netherlands


In [33]:
race_df_15.to_csv("../data/r15_netherlands_2025.csv", index=False)

In [34]:
import joblib

model  = joblib.load("../models/logreg_scaled_quali_pre_r15_netherlands.pkl")
scaler = joblib.load("../models/scaler_pre_r15_netherlands.pkl")

In [35]:
import pandas as pd

df_netherlands = pd.read_csv('../data/r15_netherlands_2025.csv')
df_netherlands.head()

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,92.309167,71.756,6,69.652,15,Netherlands
1,ALO,87.701083,69.977,12,69.366,8,Netherlands
2,ANT,87.443125,71.185,8,69.493,11,Netherlands
3,BEA,77.386529,71.113,17,70.262,19,Netherlands
4,BOR,78.776938,71.320,16,69.622,12,Netherlands


In [40]:
feature_cols = ['fp2_avg_lap', 'fp2_best_lap', 'fp2_total_laps', 'FastestQualiLap', 'QualiPosition'] 
X_netherlands = df_netherlands[feature_cols]

X_netherlands_scaled = scaler.transform(X_netherlands) 
proba_podium = model.predict_proba(X_netherlands_scaled)[:, 1]

results = df_netherlands.copy()
results["podium_probability"] = proba_podium

results_sorted = results.sort_values(by="podium_probability", ascending=False)

predicted_podium = results_sorted[["Driver", "podium_probability"]]

In [41]:
predicted_podium

,Driver,podium_probability
13,PIA,0.956182
11,NOR,0.926740
17,VER,0.855730
14,RUS,0.634839
7,HAM,0.497993
10,LEC,0.312378
1,ALO,0.221353
9,LAW,0.118730
15,SAI,0.060864
2,ANT,0.037119


In [42]:
predicted_podium.head(3)

,Driver,podium_probability
13,PIA,0.956182
11,NOR,0.926740
17,VER,0.855730
